In [5]:
import numpy as np 
import pandas as pd

In [201]:
class visitlog_transformer:
    
    def _init_(self):
        pass
    
    """funtion to impute the values on the activity column based on the obeservation click occurs on the same product while
    #product changes with the pageload
    
    # basic idea isto create session id for the users to impute the activity column
    
    1---remove the null visittime rows and save it on another df which we will concat later
    also remove the rows where both the product and activity columns are empty
    
    2---grouping by userid and finiding the diff between consecutive entries and if the difference is 
        #longer than 2 hours then marking it as different session
    
    3---create the features based on the logic that click occurs on the same product within same session   
    """
    
    def impute_activity(self,df):
        
        #step1
        
        mini_df=df.loc[df.VisitDateTime.isna()]
        df=df.loc[df.VisitDateTime.notnull()]
        mini_df=pd.concat([mini_df,df.loc[(df.ProductID.isna())&(df.Activity.isna())]],ignore_index=True)
        df=df.loc[df.filter(['ProductID','Activity']).dropna(thresh=1).index]
        
        #step2
        df=df.sort_values(['UserID','VisitDateTime']).reset_index(drop=True)
        df['time_difference']=df.groupby('UserID')['VisitDateTime'].diff(1).dt.total_seconds()/60
        new_session=((df.time_difference.isna()) | (df.time_difference > 120))
        df['session_id']=df.loc[new_session,['UserID','VisitDateTime']].groupby('UserID').rank(method='first').astype(int)
        df['session_id']=df.groupby('UserID')['session_id'].ffill()
        
        #step3
        
        df['imputed_activity']=0
        df.loc[df.Activity.isna(),'imputed_activity']=1
        tobe_imputed=(df.ProductID==df.groupby(['UserID','session_id'])['ProductID'].shift(1))&(df.Activity.isna())
        df.loc[tobe_imputed,'Activity']='CLICK'
        tobe_imputed=(df.ProductID!=df.groupby(['UserID','session_id'])['ProductID'].shift(1))&(df.Activity.isna())

        df.loc[tobe_imputed,'Activity']='PAGELOAD'        
        df=self.impute_product(df)        
        df=pd.concat([df,mini_df],ignore_index=True).reset_index(drop=True)
        
        return (df)
    
    """impute product based on the observed observation on activity fuction gets called from impute activity"""
    def impute_product(self,df):
        
        df['imputed_product']=0
        df.loc[df.ProductID.isna(),'imputed_product']=1
        df['product_shifted']=df.groupby(['UserID','session_id'])['ProductID'].shift(1)
        df.loc[(df.ProductID.isna())&(df.Activity=='CLICK'),'ProductID']=df.loc[(df.ProductID.isna())&(df.Activity=='CLICK'),'product_shifted']
        
        return (df)
    
    """   For converting Visitdatetime column to datetime column from string
    step--1#filter only the unix timestamps and set it as string after converting it to digit
    step---2 #convert the whole column to datetime column"""
    def datetime_clean(self,df):        
        #step1
        unix_times=df.VisitDateTime.astype(str).str.isdigit()
        df.loc[unix_times,'VisitDateTime']=pd.to_datetime(df.loc[unix_times,'VisitDateTime'].astype(str).str[:13],unit='ms').astype(str)
        #step2        
        df['VisitDateTime']=pd.to_datetime(df['VisitDateTime'])        
        df=df.sort_values(['UserID','VisitDateTime']).reset_index(drop=True)
        
        return (df)
        
    def string_clean(self,df):
        #cleaning the os and setting title operation inorder to capitalize the first letter    
        df['OS']=df['OS'].str.lower()
        
        df['OS']=df['OS'].str.title()
        
        #cleaning the productid column by capitalizing
        df['ProductID']=df['ProductID'].str.capitalize()
        
        #cleaning the Activity column by setting everything in upper case
        df['Activity']=df['Activity'].str.upper()
        
        return (df)
    
    def cleandata(self,df):
        
        #dropping duplicates incase if there is any        
        df=df.drop_duplicates()
        #filtering only the users with UserID
        df=df.loc[df.UserID.notnull()].reset_index(drop=True)
        
        df=self.string_clean(df)
        
        df=self.datetime_clean(df)
        
        df=self.impute_activity(df)
        
        return(df)

In [202]:
class user_transformer:    
    today_date=pd.to_datetime("2018-05-28 00:00:30.000000+00:00")
    
    def _init_(self):
        pass
        #df['today_date']
        
    def vintage_days(self,df):
        df['User_Vintage']=(pd.to_datetime("2018-05-28 00:00:30.000000+00:00")-df['Signup Date']).dt.days
        
        return(df)
    
    def cleandata(self,df):
        
        df=df.drop_duplicates('UserID')
        
        df=df.loc[df.UserID.notnull()].reset_index(drop=True)
        
        df['Signup Date']=pd.to_datetime(df['Signup Date'])
        
        df['User Segment']=df['User Segment'].fillna('unknown')
        
        return df

In [273]:
class user_visitlog_transformer:

    def _init_(self):
        pass
    
    def find_most_active_os(self,user_df,log_df):
        
        #log_df=logs_df
        
        grouped=log_df.groupby('UserID')
        
        os_mostactive=grouped['OS'].agg(pd.Series.mode).reset_index().rename(columns={'OS':'Most_Active_OS'})
        
        user_df=user_df.merge(os_mostactive,on='UserID',how='left')
        
        return(user_df)
    
    
    def recent_viewed_product(self,user_df,log_df):
        
        #log_df=logs_df
        
        log_df['is_all']=log_df.VisitDateTime.notnull()

        logmini=log_df.loc[log_df.groupby('UserID')['is_all'].transform(sum)==0]

        logmini=logmini.loc[(logmini.ProductID.notnull())]

        logmini=logmini.loc[(logmini.groupby('UserID')['ProductID'].transform('nunique')==1)&(logmini.Activity!='CLICK')]

        logmini=logmini.drop_duplicates(['UserID','ProductID'])
        
        log_df=log_df.loc[(log_df.VisitDateTime.notnull())&(log_df.Activity=='PAGELOAD')]

        log_df['is_recent']=log_df.groupby('UserID')['VisitDateTime'].transform(max)==log_df['VisitDateTime']

        #df['is_recent_sum']=df.groupby('UserID')['is_recent'].transform(sum)

        to_join=pd.concat([log_df.loc[(log_df['is_recent']==True),['UserID','ProductID']].drop_duplicates().reset_index(drop=True),logmini[['UserID','ProductID']].drop_duplicates()],ignore_index=True).rename(columns={'ProductID':'Recently_Viewed_Product'})

        user_df=user_df.merge(to_join,on='UserID',how='left')

        user_df['Recently_Viewed_Product']=user_df['Recently_Viewed_Product'].fillna('Product101')
        
        return (user_df)
    
    
    def seven_day_feats(self,user_df,log_df):
        
        #log_df=logs_df
        
        start_date=pd.to_datetime("2018-05-21")
        
        end_date=pd.to_datetime("2018-05-28")
        
        log_df=log_df.loc[(log_df.VisitDateTime >=start_date)&(log_df.VisitDateTime <end_date)]

        log_df['date']=log_df['VisitDateTime'].dt.date

        features=log_df.groupby('UserID')['date'].nunique().reset_index().rename(columns={'date':'No_of_days_Visited_7_Days'})

        features['No_of_days_Visited_7_Days']=features['No_of_days_Visited_7_Days'].fillna(0)

        features=features.merge(log_df.groupby(['UserID','Activity']).size().unstack().fillna(0).reset_index().rename(columns={'CLICK':'Clicks_last_7_days','PAGELOAD':'Pageloads_last_7_days','UNKNOWN':'Activity_unknown'})[['UserID','Clicks_last_7_days','Pageloads_last_7_days']],on='UserID',how='left')

        features[['Clicks_last_7_days','Pageloads_last_7_days']]=features[['Clicks_last_7_days','Pageloads_last_7_days']].fillna(0)
        
        user_df=user_df.merge(features,on='UserID',how='left')
        
        user_df[['No_of_days_Visited_7_Days','Clicks_last_7_days','Pageloads_last_7_days']]=user_df[['No_of_days_Visited_7_Days','Clicks_last_7_days','Pageloads_last_7_days']].fillna(0)
        
        return(user_df)
    
    def fifteen_day_feats(self,user_df,log_df):
        
        #log_df=logs_df
        
        start_date=pd.to_datetime("2018-05-13")
        
        end_date=pd.to_datetime("2018-05-28")
        
        log_df=log_df.loc[(log_df.VisitDateTime >=start_date)&(log_df.VisitDateTime <end_date)]

        features=log_df.groupby('UserID')['ProductID'].nunique().reset_index().rename(columns={'ProductID':'No_Of_Products_Viewed_15_Days'})

        features['No_Of_Products_Viewed_15_Days']=features['No_Of_Products_Viewed_15_Days'].fillna(0)
        
        user_df=user_df.merge(features,on='UserID',how='left')
        
        user_df['No_Of_Products_Viewed_15_Days']=user_df['No_Of_Products_Viewed_15_Days'].fillna(0)
        
        return(user_df)
        
        #most viewed products
        
    def most_viewed_feats(self,user_df,log_df):
            
        start_date=pd.to_datetime("2018-05-13")
        
        end_date=pd.to_datetime("2018-05-28")
        
        log_df['is_all']=True
        
        log_df=log_df.loc[(log_df.VisitDateTime >=start_date)&(log_df.VisitDateTime <end_date)]
        
        log_df=log_df.loc[(log_df.Activity=='PAGELOAD')&(log_df.ProductID.notnull())]

        log_df['no_of_views']=log_df.groupby(['UserID','ProductID'])['VisitDateTime'].transform(np.size)

        log_df['max_views']=log_df.groupby(['UserID'])['no_of_views'].transform(max)

        log_df=log_df.loc[log_df.no_of_views==log_df.max_views]

        log_df=log_df.loc[(log_df.groupby('UserID')['VisitDateTime'].transform(max)==log_df.VisitDateTime)]

        log_df=log_df.loc[log_df.groupby('UserID')['is_all'].transform('size')==1]

        user_df=user_df.merge(log_df[['UserID','ProductID']].rename(columns={'ProductID':'Most_Viewed_product_15_Days'}),on='UserID',how='left')
        
        user_df['Most_Viewed_product_15_Days']=user_df['Most_Viewed_product_15_Days'].fillna('Product101')
        
        #user_df=user_df.merge(features,on='UserID',how='left')
        
        user_df['Most_Viewed_product_15_Days']=user_df['Most_Viewed_product_15_Days'].fillna(0)
        return(user_df)
    
    
    def user_visitlog_feats(self,user_df,logs_df):
        user_transform=user_transformer()
        user_df=user_transform.cleandata(user_df)
        visitlog_transform=visitlog_transformer()
        logs_df=visitlog_transform.cleandata(logs_df)
        
        user_df=user_transform.vintage_days(user_df)
        
        user_df=self.find_most_active_os(user_df,logs_df)
        
        user_df=self.recent_viewed_product(user_df,logs_df)
        
        logs_df=logs_df.loc[logs_df.VisitDateTime.notnull()]
        
        user_df=self.seven_day_feats(user_df,logs_df)
        
        user_df=self.fifteen_day_feats(user_df,logs_df)
        
        user_df=self.most_viewed_feats(user_df,logs_df)
        
        return(user_df)

In [274]:
user_df=user_df1

log_df=log_df1

visitlog_transform=user_visitlog_transformer()

user_df=visitlog_transform.user_visitlog_feats(user_df,log_df)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [219]:
 best=pd.read_csv('../input/analytics-jobs/submission70.csv')

In [276]:
user_df=user_df.sort_values('UserID').reset_index(drop=True)

In [151]:
user_df['Most_Active_OS']=user_df['Most_Active_OS'].str.lower()

best['Most_Active_OS']=best['Most_Active_OS'].str.lower()

In [275]:
user_df=user_df.sort_values('UserID').reset_index(drop=True)

In [282]:
best['Most_Active_OS'].value_counts()

Windows                    20034
Android                     9734
Mac Os X                    2422
Linux                        920
Ios                          484
Ubuntu                       295
['Android' 'Windows']        103
Chrome Os                     14
Fedora                        12
['Ios' 'Windows']              7
['Android' 'Linux']            7
['Android' 'Mac Os X']         5
['Linux' 'Windows']            4
['Ios' 'Mac Os X']             2
['Android' 'Ubuntu']           2
['Mac Os X' 'Windows']         2
['Ubuntu' 'Windows']           1
['Android' 'Ios']              1
['Android' 'Chrome Os']        1
Name: Most_Active_OS, dtype: int64

In [254]:
user_df['Recently_Viewed_Product'].equals(best['Recently_Viewed_Product'])

True

In [255]:
user_df['Clicks_last_7_days'].equals(best['Clicks_last_7_days'])

True

In [256]:
user_df['Pageloads_last_7_days'].equals(best['Pageloads_last_7_days'])

True

In [257]:
user_df['No_Of_Products_Viewed_15_Days'].equals(best['No_Of_Products_Viewed_15_Days'])

True

In [85]:
user_df=pd.read_csv("/kaggle/input/analytics-jobs/data/data/userTable.csv")
log_df=pd.read_csv("/kaggle/input/analytics-jobs/data/data/VisitorLogsData.csv")
sample=pd.read_csv("/kaggle/input/analytics-jobs/sample_submission_M7Vpb9f.csv")